---
syncID: 
title: "Downlaoding NEON Observation Data with Python"
description: ""
dateCreated: 2020-04-24
authors: Maxwell J. Burner
contributors: Donal O'Leary
estimatedTime: 
packagesLibraries: requests, json, pandas
topics: api, data management
languagesTool: python
dataProduct: DP1.10003.001
code1: 
tutorialSeries: python-neon-api-series
urlTitle: python-neon-api-02-downloading-observational
---

In this tutorial we will learn to download Observational Sampling (OS) data from the NEON API into the Python environment.

<div id="ds-objectives" markdown="1">

### Objectives
After completing this tutorial, you will be able to:

* Navigate a NEON API request from the *data/* endpoint
* Describe the naming conventions of NEON OS data files
* Understand how to download NEON observational data using the Python Pandas library
* Describe the basic components of a Pandas dataframe


### Install Python Packages

* **requests**
* **json** 
* **numpy**
* **pandas**

We will not actually use the NumPy package in this tutorial; it is listed here because the Pandas package is built on top of NumPy, and requires that the latter be present.

</div>

In this tutorial we will learn how to download specific NEON data files into Python. We will specifically look at how to use the Pandas package to read in CSV files of observational data.

In the previous tutorial, we saw some of the data files containing information on land bird breeding counts. These are an example of NEON *observational data*. NEON has three basic types of data: Observational Sampling (OS), Instrumentation Sampling (IS), and Remote Sensing or Airborne Observation Platform data (AOP). The process for requesting data is about the same for all three, but downloading and navigating the data tends to be very different depending on which specific data product we are using.

Here we will discuss downloading observational data, as it tends to be the simplest to handle.

## Libraries Downloaded

In addition to used requests and json packages again, we will use the Pandas package to read in the data. Pandas is a library that adds data frame objects to Python, based on the data frames of the R programming language; these offer a great way to store and manipulate tabular data.

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
SERVER = 'http://data.neonscience.org/api/v0/'
SITECODE = 'TEAK'
PRODUCTCODE = 'DP1.10003.001'

## Look up Data Files

We already know from the last tutorial that landbird breeding counts (DP1.10003.001) are available at the Lower Teakettle site for 2018-06. We can again make a request to see what files in particular are available.

In [ ]:
#Make Request
data_request = requests.get(SERVER+'data/'+PRODUCTCODE+'/'+SITECODE+'/'+'2018-06')
data_json = data_request.json()

In [ ]:
#View names of files
for file in data_json['data']['files']:
    print(file['name'])

Let's take a closer look at a file name.

In [ ]:
print(data_json['data']['files'][6]['name'])

The format for most NEON data product file names is:

**NEON.D[two-digit domain number].[site code].[data product ID].[file-specific name].[date of file creation].[file extension]**

So the file whose name we singled out is domain 17, Lower Teakettle Site, Breeding Landbird point counts (DP1.10003.001), brd_perpoint.2018-06.basic, created 2019-11-07 at 15:32:35 Universal Time. The file name brd_perpoint.2018-06.basic indicates that this is the 'basic' version of bird counts by point, observed in June 2018.

Bird counts and other observational data are usually kept in CSV files in the NEON database. Often the data for a particular month-site combination will be available in through two different .csv files that represent two different 'download packages'; a 'basic' package storing only the main measurements, and an 'expanded' package that also lists the uncertainties involved in each measurement. Let's save the URL for the basic count data CSV file.

In [ ]:
#Print names and URLs of files with birdcount data
for file in data_json['data']['files']:
    if('countdata' in file['name']): #Show all files with 'countdata' in their name
        print(file['name'],file['url'])
        if('basic' in file['name']):
            bird_count_url = file['url'] #save url of file with basic bird count data


## Read file into Pandas Dataframe

There are a couple options for reading CSV files into Python. For files read directly from NEON's data repository, one popular option is the 'read_csv' function from the Pandas package. This function converts the contents of the target file into a pandas dataframe object, and has the added advantage of being able to read data files accessed through the web (Python has its own built-in package for reading CSV files, but this package can only read files present on your machine).

In [ ]:
#Read bird count CSV data into a Pandas Dataframe
df_bird = pd.read_csv(bird_count_url)

Pandas is a popular Python package for data analysis and data manipulation. The package implements dataframe objects based on the dataframes used in the R programming language, and uses these objects for storing and manipulating tabular data.

A dataframe is a two-dimensional table of data, a grid built of rows and columns of values. Generally the columns correspond to the different variables being measured, while the rows correspond to each entry or measurement taken (in this case, each bird counted). Dataframes also have a header containing labels for each column, and an index containing labels for each row; both are 'index' objects stored as attributes of the dataframe object.

Python dataframes store their contents, header, and index in different attributes of the dataframe object. Other attributes contain metadata such as the overall shape of the dataframe, and the data type of each column.

You can find more about Pandas at their [official site](https://pandas.pydata.org/), which includes a tutorials page [here](https://pandas.pydata.org/pandas-docs/version/0.15/tutorials.html).

In [ ]:
#View the column names
df_bird.columns

In [ ]:
#Print out dimensions of the new dataframe
print('Number of columns: ',df_bird.shape[1])
print('Number of Rows: ',df_bird.shape[0])

In [ ]:
#Print out names and data types of dataframe columns
print(df_bird.dtypes)

Pandas dataframes classify data as integer, floating point (decimal numbers), or object; the last category ususally indicates data stored as strings, such as text labels or date-time data.

In [ ]:
#View first five rows of dataframe using the 'head' method
df_bird.head(5)

We can now manipulate this dataframe using the various methods and functions of the Pandas library.

## Variable Information

Look again at the list of files available, specifically those that are NOT counting data.

In [ ]:
#View names of files
for file in data_json['data']['files']:
    if( (not('countdata' in file['name'])) & (not('perpoint' in file['name'])) ):
        print(file['name'])

While the .zip files are packages containing multiple bird count data tables, the remaining files mostly serve to provide context to the data. The *variables* CSV file in particular contains a dataset with information on the variables used in the count data tables. This provides useful information such as units and defintions for each variable.

In [ ]:
#Get variables information as pandas dataframe
for file in data_json['data']['files']:
    if('variables' in file['name']):
        df_variables = pd.read_csv(file['url'])

In [ ]:
#View metadata and first few rows

print('Number of rows: ', df_variables.shape[0])
print('Number of columns: ',df_variables.shape[1])

print('Data Columns:\n')
print(df_variables.dtypes)

df_variables.head(5)

The table includes a column called 'table' indicating in which file a variable appears. We want to see information on the variables for the basic bird count table, since that is the table we downloaded. We can do this using comparisons and subsetting.

In [ ]:
#Subset to view only variables in the basic countdata table
df_variables[(df_variables['table'] == 'brd_countdata')&(df_variables['downloadPkg'] == 'basic')]

### Challenge
The Pandas *concat* function takes mulitple dataframes that have the same column names and attributes, but different rows, and combines the rows from all of the input dataframes into one output dataframe. Get basic bird count data for other months at Lower Teakettle, and combine the resulting dataframes into one with *concat*.